In [1]:
from urllib.request import urlopen as uReq
import re
import requests
import bs4
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
from time import sleep
from os.path import basename

In [2]:
width_dict = {"max-width:7%;": "1", "max-width:14%;": "2", "max-width:21%;": "3", 
              "max-width:28%;": "4", "max-width:35%;": "5"} #1 being least 5 being most

In [3]:
gender_dict = {"Homme": "Male", "Femme": "Female"}

In [4]:
disease_dict = {"Acné": "Acne", "Allergie":"Allergy", "Allergie acariens":"Dust Mite Allergy", "Asthme":"Asthma",
               "Ampoules - cloques":"Blisters - blisters", "Angine de poitrine":"Angina pectoris",
               "Angoisse / trouble panique":"Anxiety", "Arrêter de fumer":"Stop smoking", "Bronchite":"Bronchitis",
               "Arthrite psoriatique":"Psoriatic arthritis", "Arthrose":"Osteoarthritis", "Brûlures":"Burns",
               "Arthrose colonne vertébrale":"Spine osteoarthritis", "Cancer du sein":"Breast cancer",
               "Céphalées":"Headache", "Cholestérol élevé":"High cholesterol", "Couperose":"Rosacea",
               "Colite ulcéreuse":"Ulcerative colitis", "Crise cardiaque":"Heart attack", "Tremblements":"Tremors",
               "Complications post-opératoires":"Post-operative complications", "Démangeaison":"Itching",
               "Cycle menstruel irrégulier":"Irregular menstrual cycle", "Défaillance nerveuse":"Nerve failure",
               "Dépression":"Depression", "Dépression post-natale":"Post-natal depression", "Thyroïde":"Thyroid",
               "Dépression chronique":"Depression", "Diabète type 1":"Type 1 diabetes", "TDAH":"ADHD",
               "Diabète type 2":"Type 2 diabetes", "Diabète type 3":"Type 3 diabetes", "Urticaire":"Urticaria",
               "Troubles d'angoisse":"Anxiety", "Trouble stress post-traumatique":"Post-traumatic stress disorder",
               "Trouble schizotypique":"Schizotypal disorder", "Trouble rythme cardiaque":"Heart rhythm disorder",
               "Trouble obsessionnel compulsif (TOC)":"Obsessive Compulsive Disorder (OCD)", "Psychose":"Psychosis",
               "Trouble bipolaire / trouble mani dépressif":"Bipolar disorder / manic depressive disorder",
               "Thyroïde enlevée":"Thyroid removed", "Tension élevée":"High tension", "Sinusite":"Sinusitis",
               "Spondylarthrite ankylosante":"Ankylosing spondylitis", "Schizophrénie":"Schizophrenia",
               "Rhume des foins":"Hay fever", "Rhumatisme":"Rheumatism", "Retour d'âge":"Return of age",
               "Pyélonéphrite":"Pyelonephritis", "Problèmes psychiques":"Psychic problems", "Otite":"Otitis",
               "Problèmes cardiaques":"Heart problems", "Pouls élevé":"High pulse", "Pneumonie":"Pneumonia",
               "Polyneuropathie":"Polyneuropathy", "Phobie sociale":"Social phobia", "Narcolepsie":"Narcolepsy",
               "MST (maladies sexuellement transmissibles)":"STDs (sexually transmitted diseases)",
               "Mise en place stérilet":"IUD insertion", "Maladie de Lyme - borréliose":"Lyme disease - borreliosis",
               "Maladie de Graves":"Graves disease", "Maladie de Crohn":"Crohn's disease", "Insomnie":"Insomnia",
               "Maladie d'Hashimoto":"Hashimoto's disease", "Maladie auto-immunitaire":"Autoimmune disease",
               "Mal de gorge":"Sore throat", "Inflammation articulaire":"Joint inflammation", "Hernie":"Hernia",
               "Infection urinaire":"Urinary tract infection", "Infection gencive":"Gum infection", "Fatigue":"Tired",
               "Infection des amygdales":"Infection of the tonsils", "Infection de la prostate":"Prostate infection",
               "Infection de la machoire":"Jaw infection", "Infection de l'estomac":"Stomach infection",
               "Infection dans la poitrine":"Chest Infection", "Infection bactérienne":"Bacterial infection",
               "Infection (plaie)":"Infection (wound)", "Glande thyroïde lente":"Slow thyroid gland",
               "Glande thyroïde hyperactive":"Overactive thyroid gland", "Fibromyalgie":"Fibromyalgia",
               "Épisode maniaque":"Manic episode", "Épilepsie":"Epilepsy", "Douleur, général":"Pain, general",
               "Douleurs menstruation":"Menstruation pain", "Douleur articulaire":"Articular pain",
               "Douleur nerveuse":"Nerve pain", "Hypothyroïdie":"Hypothyroidism", "Battements de coeur": "Heartbeats",
               "Ablation de la thyroïde":"Thyroid removal", "Pas dans la liste":"Other", "Apnée":"Apnea",
               "Tendinite épaule":"Shoulder tendinitis", "Alcoolisme":"Alcoholism", "Mal aux dents":"Toothache",
               "Syndrome des jambes sans repos":"Restless legs syndrome", "Vertige - tournis":"Vertigo - spinning",
               "Cholestérol trop élevé":"High cholesterol","Crampes jambes":"Leg cramps", "Acidité":"Acidity",
               "Effet indésirable d'un autre médicament":"Side effect of another medicine", "Attaque":"Attack",
               "Anémie suite déficience vitamine B12":"Anemia", "Défaillances cardiaques":"Cardiac failures",
               "Brûlure d'estomac - acidité gastrique":"Heartburn - gastric acidity", "Anémie":"Anemia",
               "Anticonceptie / zwangerschapspreventie":"Contraception", "Anti-conception":"Contraception", 
               "Syndrome pré-mensuel (SPM)":"Pre-monthly syndrome (PMS)", "Anémie manque de fer":"Anemia",
               "Autres problèmes de prostate":"Other prostate problems", "Endométriose":"Endometriosis",
               "Ovaires polykystiques (SOPK)":"Polycystic ovaries (PCOS)", "Allergie solaire":"Solar allergy",
               "Trouble de personnalité borderline":"Borderline personality disorder", "Pancréatite":"Pancreatitis",
               "Trouble d'angoisse":"Anxiety", "Dépression /trouble d'angoisse. borderline":"Depression",
               "Dépressif":"Depression","Dépresison , crises de panique":"Depression", "Acouphène":"Tinnitus",
               "Angoisse sociale et dépression":"Anxiety","Épuisement professionnel":"Burnout", "Stress":"Anxiety",
               "Phobie sociale, angoisses, frustration / irritation":"Anxiety", "Dépression, angoisse":"Depression",
               "Trouble personnalité cluster C":"Borderline personality disorder", "Bouche sèche":"Dry Mouth",
               "Forts changements d'humeur et dépressions":"Depression", "Sensible pour dépression":"Depression",
               "Troubles d'angoisse et de panique":"Anxiety", "Trouble d'angoisse / dépression":"Anxiety",
               "Crises de panique, dépression":"Anxiety", "SSPT, hypochondrie, neurose obsessionnelle":"hypochondria",
               "Angoisse et dépression":"Anxiety","Angoisse, crises de panique":"Anxiety", "Grippe":"Flu",
               "Surmenage / épuisement professionnel":"Burnout", "Problèmes de concentration":"ADHD",
               "Maladie de Parkinson":"Parkinson disease","Cancer de la vessie":"Bladder cancer",
               "Cancer de la peau":"Skin Cancer","Trouble d'érection":"Erectile dysfunction", "Aphtes":"Canker sores",
               "Athérosclérose":"Atherosclerosis","Cancer de la gorge":"Throat cancer", "Mal à la gorge":"Sore throat",
               "Cancer de l'estomac":"Stomach cancer", "Borstkanker":"Breast Cancer", "Oedème":"Edema",
               "Douleur musculaire":"Muscle pain","Nervosité":"Anxiety","Douleur bas du dos":"Back Pain",
               "Polyneuropathie inflammatoire aiguë":"Polyneuropathy","Problèmes de dos":"Back Pain",
               "Colite collagène":"Collagen colitis","Scarlatine":"Scarlet fever","Pharyngite":"Pharyngitis",
               "Soin de la dentition":"Teething care","Extraction dentaire":"Dental extraction",
               "Athérosclérose":"Atherosclerosis","Multiple sclérose en plaques":"Sclerosis",
               "Syndrome de Guillain-Barré":"Guillain-Barré syndrome","Douleur fantôme":"Pain, general",
               "Bourdonnement d'oreille":"Ringing in the ears","Ménopause prématurée":"Premature menopause",
               "Thrombophilie par mutation du facteur V":"Thrombophilia"}

In [5]:
le = 'https://www.meamedica.fr/glande-thyroide-hypothyroidie-a-action-lente/levothyrox'
mi = 'https://www.meamedica.fr/contraception-autre/mirena'
tr = 'https://www.meamedica.fr/douleurs-morphine/tramadol'
pa = 'https://www.meamedica.fr/depression-antidepresseurs-irs/paroxetine'
ef = 'https://www.meamedica.fr/depression-antidepresseurs-autre/effexor'
ch = 'https://www.meamedica.fr/toxicomanie/champix'
ly = 'https://www.meamedica.fr/epilepsie/lyrica'
si = 'https://www.meamedica.fr/cholesterol/simvastatine'
am = 'https://www.meamedica.fr/antibiotiques-penicillines-a-large-spectre/amoxicilline'
se = 'https://www.meamedica.fr/depression-antidepresseurs-irs/sertraline'
cy = 'https://www.meamedica.fr/depression-antidepresseurs-autre/cymbalta'
ser = 'https://www.meamedica.fr/depression-antidepresseurs-irs/seroplex'
cr = 'https://www.meamedica.fr/cholesterol/crestor'
ta = 'https://www.meamedica.fr/cancer-hormones-et-antihormones/tamoxifene'
ci = 'https://www.meamedica.fr/depression-antidepresseurs-irs/citalopram'
me = 'https://www.meamedica.fr/diabetes-medicaments-oraux/metformine'
de = 'https://www.meamedica.fr/depression-antidepresseurs-irs/deroxat'
tah = 'https://www.meamedica.fr/cholesterol/tahor'
vi = 'https://www.meamedica.fr/diabetes-medicaments-oraux/victoza'
pr = 'https://www.meamedica.fr/tension-arterielle-beta-bloquant/propranolol'
ab = 'https://www.meamedica.fr/psychose-schizophrenie-antipsychotique/abilify'
co = 'https://www.meamedica.fr/adhd-psychostimulants/concerta'
ke = 'https://www.meamedica.fr/epilepsie/keppra'
ro = 'https://www.meamedica.fr/acne/roaccutane'
lex = 'https://www.meamedica.fr/depression-antidepresseurs-irs/lexapro'
ae = 'https://www.meamedica.fr/allergies/aerius'
va = 'https://www.meamedica.fr/depression-antidepresseurs-autre/valdoxan'
ce = 'https://www.meamedica.fr/contraception-autre/cerazette'
do = 'https://www.meamedica.fr/antibiotiques-tetracyclines/doxycycline'
mer = 'https://www.meamedica.fr/systeme-immunitaire-immunosuppression/methotrexate'

In [6]:
le = [le] + [le + '/' + str(i) for i in range(2, 8)]
mi = [mi] + [mi + '/' + str(i) for i in range(2, 8)]
tr = [tr] + [tr + '/' + str(i) for i in range(2, 8)]
pa = [pa] + [pa + '/' + str(i) for i in range(2, 8)]
ef = [ef] + [ef + '/' + str(i) for i in range(2, 8)]
ch = [ch] + [ch + '/' + str(i) for i in range(2, 8)]
ly = [ly] + [ly + '/' + str(i) for i in range(2, 8)]
si = [si] + [si + '/' + str(i) for i in range(2, 8)]
am = [am] + [am + '/' + str(i) for i in range(2, 8)]
se = [se] + [se + '/' + str(i) for i in range(2, 8)]
cy = [cy] + [cy + '/' + str(i) for i in range(2, 8)]
ser = [ser] + [ser + '/' + str(i) for i in range(2, 8)]
cr = [cr] + [cr + '/' + str(i) for i in range(2, 8)]
ta = [ta] + [ta + '/' + str(i) for i in range(2, 8)]
ci = [ci] + [ci + '/' + str(i) for i in range(2, 8)]
me = [me] + [me + '/' + str(i) for i in range(2, 8)]
de = [de] + [de + '/' + str(i) for i in range(2, 8)]
tah = [tah] + [tah + '/' + str(i) for i in range(2, 8)]
vi = [vi] + [vi + '/' + str(i) for i in range(2, 8)]
pr = [pr] + [pr + '/' + str(i) for i in range(2, 8)]
ab = [ab] + [ab + '/' + str(i) for i in range(2, 8)]
co = [co] + [co + '/' + str(i) for i in range(2, 8)]
ke = [ke] + [ke + '/' + str(i) for i in range(2, 8)]
ro = [ro] + [ro + '/' + str(i) for i in range(2, 8)]
lex = [lex] + [lex + '/' + str(i) for i in range(2, 8)]
ae = [ae] + [ae + '/' + str(i) for i in range(2, 8)]
va = [va] + [va + '/' + str(i) for i in range(2, 8)]
ce = [ce] + [ce + '/' + str(i) for i in range(2, 8)]
do = [do] + [do + '/' + str(i) for i in range(2, 8)]
mer = [mer] + [mer + '/' + str(i) for i in range(2, 8)]
url = le + mi + tr + pa + ef + ch + ly + si + am + se + cy + ser + cr + ta + ci + me + de + tah + vi + pr + ab + co + ke + ro + lex + ae + va + ce + do + mer

In [7]:
rx_dose = re.compile('\((.*)\)')

rows = []
for u in url:
    html = requests.get(u).text
    soup = BeautifulSoup(html, "html.parser")
    for div_tag in soup.findAll('div', {'class': 'vote rounded btn-left'}): 
#note: there are some of these div_tag that contain boxes of text we do not want
        row = {}
        h3_tag = div_tag.find('h3')
        if h3_tag is None:
            continue #removes all the irrelevant <div class>:'vote rounded btn-left'
        row['medicine'] = h3_tag.text.strip() #Picks up medicine name and assigns to row of that name
        subtext_tag = div_tag.find('div', {'class': 'subText'}) 
        #div_tag contains multiple elements we want to store in different columns
        subText = subtext_tag.text.split('\r\n') #use the \ to seperate the information
        var = subText[1].strip().split(' | ') #use the | to seperate the information
        row['date'] = var[0] #takes first value in var
        row['gender'] = var[1] #takes second value in var
        row['age'] = var[2] if len(var)==3 else np.nan #takes third value in var if it's not empty
        match = rx_dose.search(subText[3]) #takes fourth element of subText
        row['dose'] = np.nan if match is None else match.group(1) #takes 2nd element of match if its not empty
        row['disease'] = subText[4].strip() if len(subText)>4 else np.nan #takes 4th element of subText if not empty
        span_tag = div_tag.find('span', {'class': 'initial-text'}) #finds up the review in its div_tag
        row['comment'] = span_tag.text if span_tag else '' #takes content of span_tag if there is any
        for span_tag in div_tag.findAll('span', {'style': 'width:65%'}): #finds the width % of the 4 little graphs
            text = span_tag.text
            img_tag = span_tag.next_sibling.next_sibling
            row[text] = img_tag.attrs['style'] if img_tag else ''
        rows.append(row)

df = pd.DataFrame(rows)
df['age']=df['age'].astype(float)
df['age_range']=pd.cut(df['age'], [0,20,35,50,65,80,np.inf], labels = ['0-20','21-35','36-50','51-65','66-80','80+'])
df['gender'].replace(gender_dict, inplace=True)
df['disease'].replace(disease_dict, inplace=True)
df["Efficacité"].replace(width_dict, inplace=True)
df["Quantité effets secondaires"].replace(width_dict, inplace=True) 
df["Gravité effets secondaires"].replace(width_dict, inplace=True)
df["Facilité d'emploi"].replace(width_dict, inplace=True)

In [143]:
df = df.rename(columns={"Efficacité": "efficiency", "Quantité effets secondaires": "quantity_of_secondary_effects", 
                   "Gravité effets secondaires": "severity_of_secondary_effects", "Facilité d'emploi": "ease_of_use"})

In [146]:
df.to_csv('all_reviews_v3.csv') #we store the dataframe in a csv file